In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.datasets as dataset
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import numpy as np
import os
%matplotlib qt

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [3]:
# 超参数
num_input = 513*151
in_channels = 1
features = 8
num_hidden = 200
num_class = 6
batch_size = 128
learning_rate = 0.005
epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# 读取数据集
data_path = "D:\\Data_Set\\pyVHR\\MovingClass\\Class2\\Data\\"
label_path = "D:\\Data_Set\\pyVHR\\MovingClass\\Class2\\Labels\\"
data_files = os.listdir(data_path)
label_files = os.listdir(label_path)
all_data = []
all_label = []
for i in range(len(data_files)):
    data = np.loadtxt(data_path + data_files[i])
    all_data.append(data)
    label = np.loadtxt(label_path + label_files[i])
    all_label.append(label)

In [5]:
all_data = np.array(all_data)
all_label = np.array(all_label, dtype=np.int)

C:\Users\WuHP\AppData\Local\Temp\ipykernel_22292\1104655312.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all_label = np.array(all_label, dtype=np.int)


In [6]:
count = len(all_label)
train = [i for i in range(count)]
test = []
for i in range(0, count, 4):
    test.append(i)
    train.remove(i)
x_train = all_data[train]
y_train = all_label[train]
x_valid = all_data[test]
y_valid = all_label[test]

In [7]:
# 数据预处理
train_transform = transforms.Compose([transforms.ToTensor()])
test_transform = transforms.Compose([transforms.ToTensor()])

In [8]:
# 构建Dataset类训练、测试数据， 集成Dataset，复写__getitem__
class MyDataset(Dataset):
    def __init__(self, data, label, transform):
        '''
        自制数据集的Dataset
        '''
        self.data = data
        self.labels = label
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # 返回第index个数据样本
        img = self.data[index]
        label = self.labels[index]
        if self.transform is not None:
            img = self.transform(img)

        return img, label

train_data = MyDataset(x_train, y_train, train_transform)
Train_loader =DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_data = MyDataset(x_valid, y_valid, test_transform)
Test_loader = DataLoader(dataset=test_data, batch_size=len(test_data), shuffle=False)

In [9]:
# 搭建CNN网络
class CNN(nn.Module):
    def __init__(self, in_channel=1, feature=8, num_class=2):
        super(CNN, self).__init__()
        self.Conv1 = nn.Conv2d(in_channels=in_channel, out_channels=feature, kernel_size=3, stride=1, padding=1)     # 卷积层1： Batch * 8 * 513 * 151
        self.Relu1 = nn.ReLU()                               # 激活层
        self.Pool1 = nn.MaxPool2d(kernel_size=2, stride=2)                                                           # 最大池化1：Batch * 8 * 256 * 75

        self.Conv2 = nn.Conv2d(feature, feature, kernel_size=3, stride=1, padding=1)                                 # 卷积层2：  Batch * 8 * 256 * 75
        self.Relu2 = nn.ReLU()                               # 激活层
        self.Pool2 = nn.MaxPool2d(kernel_size=2, stride=2)                                                           # 最大池化2：Batch * 8 * 128 * 37

        self.Conv3 = nn.Conv2d(feature, feature // 2, kernel_size=3, stride=1, padding=1)                            # 卷积层3：Batch * 4 * 128 * 37
        self.Relu3 = nn.ReLU()                               # 激活层
        self.Pool3 = nn.MaxPool2d(kernel_size=2, stride=2)                                                           # 最大池化3：Batch * 4 * 64 * 18

        self.Conv4 = nn.Conv2d(feature//2, 1, kernel_size=3, stride=1, padding=1)                                    # 卷积层4：Batch * 1 * 64 * 18
        self.Relu4 = nn.ReLU()                               # 激活层
        self.Pool4 = nn.MaxPool2d(kernel_size=2, stride=2)                                                           # 最大池化4 Batch * 1 * 32 * 9

        self.Fc = nn.Linear(32*9, num_class)               # 全连接层输出

    # 参数前向传播
    def forward(self, x):
        x = self.Conv1(x)
        x = self.Relu1(x)
        x = self.Pool1(x)
        x = self.Conv2(x)
        x = self.Relu2(x)
        x = self.Pool2(x)
        x = self.Conv3(x)
        x = self.Relu3(x)
        x = self.Pool3(x)
        x = self.Conv4(x)
        x = self.Relu4(x)
        x = self.Pool4(x)
        x = x.reshape(x.shape[0], -1)   # [Batch, , , ] --> [Batch, ]
        x = self.Fc(x)
        return x

# 初始化模型
model = CNN(in_channel=in_channels, feature=features, num_class=2).to(device)

In [10]:
#定义损失函数和优化器
LossFuc = nn.CrossEntropyLoss()     # 该损失函数包括了softmax，因此不需要在输出层额外添加softmax激活函数
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
# 训练
loss_lst = []
epochs = 30
for epoch in range(epochs):
    loss_epoch = 0.0
    for batch_index, (images, labels) in enumerate(Train_loader):
        images = images.to(torch.float32)
        images  = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        # 计算损失
        loss = LossFuc(outputs, labels.long())
        loss_epoch += loss

        # 梯度后向传播
        optimizer.zero_grad()   # 置零
        loss.backward()         # 向后传
        optimizer.step()        # 更新

        # if batch_index % 100 == 0:
        print('[{}/{}], [{}/{}], loss={:.4f}'.format(epoch+1, epochs, batch_index, len(Train_loader), loss))
    loss_lst.append(loss_epoch / (batch_index + 1))

[1/30], [0/24], loss=0.0230
[1/30], [1/24], loss=0.0102
[1/30], [2/24], loss=0.0122
[1/30], [3/24], loss=0.0093
[1/30], [4/24], loss=0.0024
[1/30], [5/24], loss=0.0043
[1/30], [6/24], loss=0.0030
[1/30], [7/24], loss=0.0060
[1/30], [8/24], loss=0.0144
[1/30], [9/24], loss=0.0145
[1/30], [10/24], loss=0.0072
[1/30], [11/24], loss=0.0057
[1/30], [12/24], loss=0.0081
[1/30], [13/24], loss=0.0103
[1/30], [14/24], loss=0.0049
[1/30], [15/24], loss=0.0103
[1/30], [16/24], loss=0.0078
[1/30], [17/24], loss=0.0115
[1/30], [18/24], loss=0.0118
[1/30], [19/24], loss=0.0039
[1/30], [20/24], loss=0.0023
[1/30], [21/24], loss=0.0088
[1/30], [22/24], loss=0.0026
[1/30], [23/24], loss=0.0000
[2/30], [0/24], loss=0.0034
[2/30], [1/24], loss=0.0041
[2/30], [2/24], loss=0.0089
[2/30], [3/24], loss=0.0009
[2/30], [4/24], loss=0.0035
[2/30], [5/24], loss=0.0034
[2/30], [6/24], loss=0.0030
[2/30], [7/24], loss=0.0128
[2/30], [8/24], loss=0.0023
[2/30], [9/24], loss=0.0022
[2/30], [10/24], loss=0.0063
[2/30

In [26]:
loss_data = [i.data.cpu().item() for i in loss_lst]
loss_data

[0.008100616745650768,
 0.0043912469409406185,
 0.0038275287952274084,
 0.00430113822221756,
 0.0037740988191217184,
 0.0027319069486111403,
 0.001847862615250051,
 0.001686320174485445,
 0.0016025400254875422,
 0.0018242374062538147,
 0.0019270607735961676,
 0.040538933128118515,
 0.21139827370643616,
 0.04039347916841507,
 0.018875114619731903,
 0.012045796029269695,
 0.00857715867459774,
 0.0067106811329722404,
 0.005308560095727444,
 0.004519590176641941,
 0.00440954277291894,
 0.004178288858383894,
 0.0033514010719954967,
 0.0025760724674910307,
 0.0019516320899128914,
 0.0016667889431118965,
 0.0014591312501579523,
 0.0014677181607112288,
 0.0012731601018458605,
 0.001169492257758975]

In [27]:
plt.plot(np.arange(0, epochs), np.array(loss_data))
plt.xticks([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
plt.ylabel("Loss", fontproperties='Times New Roman', fontsize=18)
plt.xlabel("训练轮次", fontproperties='SimSun', fontsize=18)
# plt.savefig(r"D:\OneDrive\桌面\绘图1.svg", dpi=500)

Text(0.5, 0, '训练轮次')

In [28]:
with torch.no_grad():
    correct_num = 0
    total_num = 0
    for images, labels in Test_loader:
        images = images.to(torch.float32)
        images  = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predictions = torch.max(outputs, 1)
        correct_num = (predictions==labels).sum()
        total_num = (predictions.size(0))
        print("测试集精度为：{}%".format(correct_num/total_num*100))

测试集精度为：98.88211059570312%


In [29]:
labels

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,

In [33]:
t_count = 0
f_count = 0
with torch.no_grad():
    t_f_num = 0
    f_t_num = 0
    t_t_num = 0
    f_f_num = 0
    i = 0
    for images, labels in Test_loader:
        for i in range(len(images)):
            image = images[i].to(torch.float32).to(device)
            label = labels[i]
            output = model(image)
            _, prediction = torch.max(output, 1)
            if label == 1:
                t_count += 1
                if prediction == 1:
                    t_t_num += 1
                elif prediction == 0:
                    t_f_num += 1
            if label == 0:
                f_count += 1
                if prediction == 0:
                    f_f_num += 1
                elif prediction == 1:
                    f_t_num += 1
        print(len(images))

984


In [34]:
test_count = len(images)
print(test_count, t_count, t_f_num)
print(test_count, t_count, t_t_num)
print(test_count, f_count, f_t_num)
print(test_count, f_count, f_f_num)

984 645 7
984 645 638
984 339 4
984 339 335


In [70]:
confun_matrix = np. array([[t_t_num, t_f_num], [f_t_num, f_f_num]])
# confun_matrix = np. array([[t_t_num / t_count, t_f_num / t_count], [f_t_num / f_count, f_f_num / f_count]])
confun_matrix = np.around(confun_matrix, decimals=4)
confun_matrix

array([[638,   7],
       [  4, 335]])

In [74]:
plt.figure(figsize=(13,8))
# plt.text(-0.2,0, str(confun_matrix[0,0]), fontsize=22, fontdict={'color': 'white'})
# plt.text(-0.2,1, str(confun_matrix[1,0]), fontsize=22)
# plt.text(0.8,0, str(confun_matrix[0,1]), fontsize=22)
# plt.text(0.8,1, str(confun_matrix[1,1]), fontsize=22, fontdict={'color': 'white'})
plt.text(-0.1,0, str(confun_matrix[0,0]), fontsize=22, fontdict={'color': 'white'})
plt.text(0,1, str(confun_matrix[1,0]), fontsize=22)
plt.text(1,0, str(confun_matrix[0,1]), fontsize=22)
plt.text(0.9,1, str(confun_matrix[1,1]), fontsize=22)
plt.xticks([0, 1], ['周期', '非周期'], fontproperties='SimSun', fontsize=18)
plt.yticks([0, 1], ['周期', '非周期'], fontproperties='SimSun', fontsize=18)
plt.xlabel("预测值", fontproperties='SimSun', fontsize=18)
plt.ylabel("真实值", fontproperties='SimSun', fontsize=18)
plt.imshow(confun_matrix, cmap='Blues')
plt.tight_layout()
plt.colorbar()
plt.savefig(r"D:\OneDrive\桌面\网络训练混淆矩阵.svg", dpi=500)

AttributeError: module 'tensorflow' has no attribute 'confusion_matrix'

In [37]:
torch.save(model, 'MotionClassTwoCNN2024.pth')

In [28]:
tt = torch.load('MotionClassTwoCNN.pth', map_location=device)
tt.eval()

CNN(
  (Conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Relu1): ReLU()
  (Pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Relu2): ReLU()
  (Pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Relu3): ReLU()
  (Pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Fc): Linear(in_features=9216, out_features=2, bias=True)
)

In [33]:
# pre1 = x_valid[77]
pre1 = np.loadtxt("D:\\Data_Set\\pyVHR\\MovingClass\\Data\\1205_1.txt")
print(y_valid[77])

1


In [34]:
tran = transforms.ToTensor()
pre1 = tran(pre1).to(device)
pre1 = pre1.view(1,1,513,151).to(torch.float32)

In [35]:
predict = tt(pre1)
predict

tensor([[-11.1065,  11.0189]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [36]:
print(int(predict.argmax()))

1
